# Testing the Pretrained Retrained Model

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-Up-Imports" data-toc-modified-id="Set-Up-Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set-Up Imports</a></span></li><li><span><a href="#Load-Models" data-toc-modified-id="Load-Models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Models</a></span><ul class="toc-item"><li><span><a href="#Load-Pretrained-Tokenizer" data-toc-modified-id="Load-Pretrained-Tokenizer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Pretrained Tokenizer</a></span></li><li><span><a href="#Load-Trained-Model" data-toc-modified-id="Load-Trained-Model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load Trained Model</a></span></li><li><span><a href="#Create-Pipeline" data-toc-modified-id="Create-Pipeline-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create Pipeline</a></span></li></ul></li><li><span><a href="#Testing-metaverse-politics-disinformation-society.txt" data-toc-modified-id="Testing-metaverse-politics-disinformation-society.txt-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Testing <code>metaverse-politics-disinformation-society.txt</code></a></span><ul class="toc-item"><li><span><a href="#Load-Dataset" data-toc-modified-id="Load-Dataset-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load Dataset</a></span></li><li><span><a href="#Testing-the-Trained-Model" data-toc-modified-id="Testing-the-Trained-Model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Testing the Trained Model</a></span></li><li><span><a href="#Annotate" data-toc-modified-id="Annotate-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Annotate</a></span></li></ul></li><li><span><a href="#Overkill" data-toc-modified-id="Overkill-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Overkill</a></span></li></ul></div>

## Set-Up Imports

In [13]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import PreTrainedModel
from transformers.pipelines.pt_utils import KeyDataset
import torch

from tqdm.auto import tqdm

from datasets import Dataset, DatasetDict, load_dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

import pandas as pd
import numpy as np
import logging
from glob import glob
from os import path
import functools

from IPython.display import HTML, display

import spacy
from spacy import displacy

## Load Models

### Load Pretrained Tokenizer

In [14]:
model_checkpoint = "distilbert-base-uncased"
category_codes = dict(enumerate(['Claim', 'Concluding Statement', 'Counterclaim', 'Evidence', 'Lead', 'Position', 'Rebuttal']))
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

### Load Trained Model

In [15]:
model_path = r"models_gitignored/distilbert-base-uncased-finetuned-sentence-classification/checkpoint-12626"
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path, id2label=category_codes)
loaded_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

### Create Pipeline

In [16]:
pipe = pipeline("text-classification", model=loaded_model, tokenizer=tokenizer)
pipe

## Testing `metaverse-politics-disinformation-society.txt`

### Load Dataset

In [8]:
articles_dataset = load_dataset("text", data_files={"valid":[r"datagen/nytimes/metaverse-politics-disinformation-society.txt"]})['valid']

# filter out empty strings
articles_dataset = articles_dataset.filter(lambda example: example['text'])

articles_dataset[1]

Using custom data configuration default-af442e383791019f
Reusing dataset text (C:\Users\Prannaya\.cache\huggingface\datasets\text\default-af442e383791019f\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Prannaya\.cache\huggingface\datasets\text\default-af442e383791019f\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4\cache-e4ae27236b288068.arrow


{'text': 'In the metaverse, a user might curate a digital avatar, like a character in a video game. Through the eyes of their avatar, they would experience a digital reality as active and engaging as the physical one. Some futurists believe that soon we might attend doctor’s appointments or class there.'}

### Testing the Trained Model

In [10]:
lst = []
for i, out in enumerate(tqdm(pipe(KeyDataset(articles_dataset, "text")))):
    print(articles_dataset[i]['text'], out)
    lst.append((articles_dataset[i]['text'], out))

  0%|          | 0/21 [00:00<?, ?it/s]

The metaverse is coming. It was once a science-fiction fantasy, most notably in Neal Stephenson’s novel “Snow Crash,” of an all-encompassing virtual universe that would exist alongside the physical one. But technological advances have brought this transformation of human society close enough to reality to demand that we consider its consequences. {'label': 'Lead', 'score': 0.9138175845146179}
In the metaverse, a user might curate a digital avatar, like a character in a video game. Through the eyes of their avatar, they would experience a digital reality as active and engaging as the physical one. Some futurists believe that soon we might attend doctor’s appointments or class there. {'label': 'Evidence', 'score': 0.4578322768211365}
But while the metaverse could revolutionize work and play, it is essential to remain wary of the dangers that will emerge if it subsumes daily life. {'label': 'Rebuttal', 'score': 0.8579508066177368}
Virtual environments will supercharge disinformation campa

### Annotate

In [11]:
def displayAnnots(lst):
    test = pd.DataFrame([{"text":i, **j} for i, j in lst])
    texts = []
    labels = []
    for text, label in test.loc[:, :"label"].values:
        if len(labels):
            if labels[-1] == label:
                texts[-1] += " "+text
                continue

        texts.append(text)
        labels.append(label)

    df = pd.DataFrame({"text":texts, "label":labels})
    ents = functools.reduce(lambda lst, new: lst+[lst[-1]+len(new)], df.text, [0])
    df["start"] = ents[:-1]
    df["end"] = ents[1:]
    doc = {
        "text": df.text.sum(),
        "ents": [dict(start=int(row["start"]), end=int(row["end"]), label=row["label"]) for i, row in df.iterrows()],
    }

    labels = ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal']
    colors = ["#ACDDDE", "#CAF1DE", "#E1F8DC", "#FEF8DD", "#FFE7C7", "#F7D8BA", "#D6CDEA"]
    options = {"ents": labels, "colors": dict(zip(labels, colors))}
    displacy.render(doc, style="ent", options=options, manual=True, jupyter=True)
    print('\n\n')

displayAnnots(lst)

## Overkill

In [12]:
for filename in glob("datagen/nytimes/*.txt"):
    try:
        display(HTML("<h2>"+filename.split("/")[-1]+"</h2>"))
        articles_dataset = load_dataset("text", data_files={"valid":[filename]})['valid'].filter(lambda example: example['text'])
        lst = []
        for i, out in enumerate(tqdm(pipe(KeyDataset(articles_dataset, "text")))):
            print(articles_dataset[i]['text'], out)
            lst.append((articles_dataset[i]['text'], out))

        displayAnnots(lst)
    except UnicodeDecodeError: print("UnicodeDecodeError")

Using custom data configuration default-c7f59a738d9c36bc
Reusing dataset text (C:\Users\Prannaya\.cache\huggingface\datasets\text\default-c7f59a738d9c36bc\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Prannaya\.cache\huggingface\datasets\text\default-c7f59a738d9c36bc\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4\cache-de0874ee69439509.arrow


  0%|          | 0/11 [00:00<?, ?it/s]

In 2019 a wave of anti-abortion laws swept this country — a common enough event in the United States, where hundreds of such laws have passed during the last decade. But these grabbed the public’s attention in a way many others hadn’t. Georgia banned abortion after about six weeks of pregnancy, or about two weeks after a missed menstrual period. Ohio, Mississippi, Louisiana and Kentucky did the same, while Missouri banned the procedure at eight weeks. Alabama went the furthest, banning virtually all abortions in the state. {'label': 'Evidence', 'score': 0.9837203025817871}
Though most of these laws were quickly blocked by the courts — they were obviously unconstitutional under Roe v. Wade — the backlash to their passing was intense, especially in Georgia, a major hub of film and television production. Boycotts were threatened. Netflix and Disney spoke out. The actress Alyssa Milano even tried to get a “Lysistrata”-style sex strike off the ground. {'label': 'Evidence', 'score': 0.934319

...

Dataset text downloaded and prepared to C:\Users\Prannaya\.cache\huggingface\datasets\text\default-8f4806c7926efb6b\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?it/s]

The remnants of two storm systems that converged in drought-stricken California this week helped pump new life into one of Yosemite National Park’s most popular attractions: Yosemite Falls. {'label': 'Evidence', 'score': 0.6039536595344543}
“We had quite a storm,” the park said in an Instagram post on Tuesday, after more than six inches of rain fell over a 36-hour period. The park encompasses more than 747,000 acres along the central western slopes of the Sierra Nevada mountain range in east-central California. {'label': 'Evidence', 'score': 0.9756234884262085}
Although the park said it did not have direct measurements, it said that “sensors suggest a few feet of snow fell at the higher elevations,” and that the snow level for much of the storm “was high,” which caused rivers and creeks “to rise substantially.” The Merced River at the Pohono Bridge rose 8.5 feet, the park said, noting it was 1.5 feet below flood stage. {'label': 'Evidence', 'score': 0.9526221752166748}
At 2,425 feet hi